In [20]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, plot_bar, plot_heatmap, plot_corr_triangle, plot_cdf
from core.s3 import S3AssetManager

In [21]:
notebook_name = "el_dorado_rendering"
s3 = S3AssetManager(notebook_name=notebook_name)

In [22]:
microbiologia = s3.read_csv("raw/dorado/rendering_microbiologia/microbiologia_el_dorado.csv")
evi = s3.read_csv("raw/dorado/rendering_microbiologia/evicerado_el_dorado.csv")


microbiologia["date"] = pd.to_datetime(microbiologia["date"])
microbiologia["month"] = microbiologia["date"].dt.to_period("M")
m = microbiologia["month"].dt.to_timestamp()
microbiologia ["month"] = m.dt.strftime("%b-%Y")
cats = pd.date_range(m.min(), m.max(), freq="MS").strftime("%b-%Y")
microbiologia["month"] = pd.Categorical(microbiologia["month"], categories=cats, ordered=True)


evi["fecha"] = pd.to_datetime(evi["fecha"])
evi["month"] = evi["fecha"].dt.to_period("M")
m = evi["month"].dt.to_timestamp()
evi ["month"] = m.dt.strftime("%b-%Y")
cats = pd.date_range(m.min(), m.max(), freq="MS").strftime("%b-%Y")
evi["month"] = pd.Categorical(evi["month"], categories=cats, ordered=True)

In [23]:

micro_month = microbiologia.groupby(["month", "etapa", "microorganismo"]).agg(
    prev=("have_micro", "mean"),
    result=('result', 'mean'),
    ph=('ph', 'mean'),
    orp=('orp', 'mean'),
    cloro=('cloro', 'mean'),
    n_analysis=('have_micro', 'count')
    ).reset_index()
micro_month["prev"] = micro_month["prev"] * 100


evi_group = evi.groupby("month").agg(
                    pollos_remisionados=('pollos_remisionados', "sum"),
                    buches_eviceracion=('buches_zona_eviceración', "sum"),
                    intestinos_eviceracion=('intestinos_zona_eviceración', "sum"),
                    gr_material_fecal=('gr_materia_fecal', "sum"),
           ).reset_index()
evi_group["kg_descuento"] = (
evi_group["buches_eviceracion"]
+ evi_group["intestinos_eviceracion"])

evi_group

df_kilos = evi_group[["month", "kg_descuento"]].copy()
df_kilos["etapa"] = "Descuento (Kg)"
df_kilos = df_kilos.rename(columns={"kg_descuento": "prev"})
df_kilos

micro_month_salmonella = micro_month[micro_month["microorganismo"] == "Salmonella"]
micro_month_campy = micro_month[micro_month["microorganismo"] == "Campylobacter"]
df_salmo = pd.concat([micro_month_salmonella, df_kilos], ignore_index=True)
df_campy = pd.concat([micro_month_campy, df_kilos], ignore_index=True)

In [24]:

fig = plot_line(
    df=df_salmo,
    x_col="month",
    y_col="prev",
    group_col="etapa",
    secondary_y_col="Descuento (Kg)",
    secondary_y_title="Kilogramos",
    x_title="Mes",
    y_title="Prevalencia (%)",
    title="Prevalencia Salmonella vs Kilos de Descuento",
    line_colors={
        "Poll Ext Visceras": "#1f4e5f",
        "ciegos": "#3d8b7a",
        "salida chiller": "#9db494",
        "prechiller": "#f2b47e",
        "Descuento (Kg)": "#ef8a82"
    },
    height = 500,
    width = 900,
)

fig.show()
s3.save_plotly_html(fig, "salmonella_line.html")

In [25]:

fig = plot_line(
    df=df_campy,
    x_col="month",
    y_col="prev",
    group_col="etapa",
    secondary_y_col="Descuento (Kg)",  # <--- Esto activa el eje derecho
    secondary_y_title="Kilogramos",
    x_title="Mes",
    y_title="Prevalencia (%)",
    title="Prevalencia Campylobacter vs Kilos de Descuento",
    line_colors={
        "Poll Ext Visceras": "#1f4e5f",
        "ciegos": "#3d8b7a",
        "salida chiller": "#9db494",
        "prechiller": "#f2b47e",
        "Descuento (Kg)": "#ef8a82" # Color salmón para el eje derecho
    },
    height = 500,
    width = 900,
)

fig.show()
s3.save_plotly_html(fig, "campy_line.html")

In [26]:
micro_month

,month,etapa,microorganismo,prev,result,ph,orp,cloro,n_analysis
0,Jan-2025,Poll Ext Visceras,Campylobacter,46.666667,4130.600000,NaN,NaN,NaN,15
1,Jan-2025,Poll Ext Visceras,Salmonella,46.666667,607.933333,NaN,NaN,NaN,15
2,Jan-2025,ciegos,Campylobacter,56.410256,137270.846154,NaN,NaN,NaN,39
3,Jan-2025,ciegos,Salmonella,42.500000,9445.925000,NaN,NaN,NaN,40
4,Jan-2025,prechiller,Campylobacter,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
75,Oct-2025,ciegos,Salmonella,37.500000,348.812500,NaN,NaN,NaN,16
76,Oct-2025,prechiller,Campylobacter,50.000000,829.000000,NaN,NaN,NaN,4
77,Oct-2025,prechiller,Salmonella,0.000000,0.000000,NaN,NaN,NaN,4
78,Oct-2025,salida chiller,Campylobacter,50.000000,196.125000,5.92750,422.875000,4.100000,8


In [29]:
last_month=micro_month["month"].max()
fig =plot_heatmap(
    df=micro_month[micro_month["month"] == last_month],
    x_col="etapa",
    y_col="microorganismo",
    value_col="prev",
    secondary_col="n_analysis",
    secondary_aggfunc="sum",
    aggfunc="mean",
    x_order= ["ciegos", "Poll Ext Visceras", "prechiller", "salida chiller"],
    value_unit="%",
    decimals_value=1,
    unit_position="suffix",
    secondary_prefix="",
    #colorscale="Blues",
    title=f"<b>Prevalencias y número de análisis en {last_month}</b>",
    show_secondary_labels=True,
    secondary_position="bottom",
    font_color="black",
    center_font_size=14,
    secondary_font_size=10,
    width=800,
    height=300,
    transparent_bg=True,
)
fig.show()
s3.save_plotly_html(fig, "prevalencia_heatmap.html")

In [36]:

for microorganismo in microbiologia["microorganismo"].unique():
    cond_microorganismo = microbiologia["microorganismo"] == microorganismo
    cond_last_month = microbiologia["month"] == microbiologia["month"].max()

    last_month_microorganismo = microbiologia[cond_microorganismo & cond_last_month].groupby(["etapa", "granja"]).agg(
    prev=("have_micro", "mean"),
    result=('result', 'mean'),
    ph=('ph', 'mean'),
    orp=('orp', 'mean'),
    cloro=('cloro', 'mean'),
    n_analysis=('microorganismo', 'count')
    ).reset_index()
    last_month_microorganismo["prev"] = last_month_microorganismo["prev"] * 100

    f = plot_heatmap(
        df=last_month_microorganismo,
        x_col="etapa",
        y_col="granja",
        value_col="prev",
        secondary_col="n_analysis",
        secondary_aggfunc="sum",
        decimals_value=1,
        aggfunc="mean",
        x_order= ["ciegos", "Poll Ext Visceras", "prechiller", "salida chiller"],
        value_unit="%",
        unit_position="suffix",
        secondary_prefix="",
        #colorscale="Blues",
        title=f"<b>Prevalencia de {microorganismo} en {last_month} por granja</b>",
        show_secondary_labels=True,
        secondary_position="bottom",
        font_color="black",
        center_font_size=14,
        secondary_font_size=10,
        width=800,
        height=500,
        transparent_bg=True,
    )
    f.show()
    name = f"prev_last_month_{microorganismo}.html"
    print(name)
    s3.save_plotly_html(f, name)


prev_last_month_Salmonella.html


prev_last_month_Campylobacter.html


In [31]:
microbiologia
cond_chiller = microbiologia["etapa"] == "salida chiller"
cond_last_month = microbiologia["month"] == microbiologia["month"].max()
chiller_last_month = microbiologia[cond_chiller]

index_cols = ['ph', 'orp', 'cloro']

df_pivot = chiller_last_month.pivot_table(
    index=index_cols,
    columns='microorganismo',
    values=['log_result', 'have_micro'],
    aggfunc='mean'
)

df_pivot.columns = [f'{val}_{micro.lower()}' for val, micro in df_pivot.columns]

df_final = df_pivot.reset_index()
df_final

,ph,orp,cloro,have_micro_campylobacter,have_micro_salmonella,log_result_campylobacter,log_result_salmonella
0,5.02,486.0,4.4,NaN,0.0,NaN,0.00000
1,5.09,503.0,4.3,NaN,0.0,NaN,0.00000
2,5.09,510.0,4.4,NaN,0.0,NaN,0.00000
3,5.43,400.0,4.3,NaN,0.0,NaN,0.00000
4,5.62,522.0,4.1,NaN,0.0,NaN,0.00000
...,...,...,...,...,...,...,...
645,7.40,422.0,4.0,1.0,0.0,1.991226,0.00000
646,7.46,363.0,4.2,0.0,0.0,0.000000,0.00000
647,7.47,337.0,4.1,1.0,1.0,1.544068,2.00000
648,7.51,327.0,4.5,1.0,0.0,0.602060,0.00000


In [32]:
fig = plot_corr_triangle(
    df_final,value_cols=["ph", "have_micro_campylobacter","cloro", "orp", "have_micro_salmonella"], method='pearson', 
    title='<b>Correlación (Pearson) — triángulo superior</b>',
     decimals=2, width=1000, height=400, 
     )
fig.show()
s3.save_plotly_html(fig, "correlacion_chiller.html")

In [12]:
microbiologia

,date,granja,year_month,have_micro,result,log_result,lot,galpon_genero,ph,orp,cloro,etapa,microorganismo,month
0,2025-01-02,Bellavista,2025-01,0,0.0,0.000000,2.0,"Galpón 8, Macho",5.89,516.0,4.4,salida chiller,Salmonella,Jan-2025
1,2025-01-03,Bellavista,2025-01,0,0.0,0.000000,3.0,"Galpón 1, Macho",5.91,504.0,4.4,salida chiller,Salmonella,Jan-2025
2,2025-01-04,Villa juliana,2025-01,0,0.0,0.000000,4.0,"Galpón 2, Macho",6.01,469.0,4.6,salida chiller,Salmonella,Jan-2025
3,2025-01-06,Villa juliana,2025-01,1,28.0,1.447158,6.0,"Galpón 4, Macho",6.03,499.0,4.3,salida chiller,Salmonella,Jan-2025
4,2025-01-07,Villa juliana,2025-01,1,642255.0,5.807707,7.0,"Galpón 3, Hembra",5.92,497.0,4.2,salida chiller,Salmonella,Jan-2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842,2025-09-29,Pollilandia ii,2025-09,0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,prechiller,Campylobacter,Sep-2025
1843,2025-10-01,La esperanza,2025-10,1,3314.0,3.520353,NaN,NaN,NaN,NaN,NaN,prechiller,Campylobacter,Oct-2025
1844,2025-10-03,Villa juliana,2025-10,0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,prechiller,Campylobacter,Oct-2025
1845,2025-10-06,Villa juliana,2025-10,0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,prechiller,Campylobacter,Oct-2025


In [37]:
fig = plot_cdf(chiller_last_month, value_col='ph', group_col='month', 
width=800, height=500)
s3.save_plotly_html(fig, "cdf_ph.html")


In [38]:

fig = plot_cdf(chiller_last_month, value_col='orp', group_col='month', width=800, height=500)
s3.save_plotly_html(fig, "cdf_orp.html")

In [39]:
fig = plot_cdf(chiller_last_month, value_col='cloro', group_col='month', width=800, height=500)
s3.save_plotly_html(fig, "cdf_cloro.html")